In [ ]:
from google.colab import files
uploaded = files.upload()

Saving u.data to u.data
Saving u.item to u.item


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load the uploaded files directly
movies = pd.read_csv('https://raw.githubusercontent.com/KaviyaGnanavel/Movie-reccomendation/refs/heads/main/u.item', sep='|', encoding='latin-1', header=None, usecols=[0, 1], names=['movie_id', 'title'])
ratings = pd.read_csv('https://github.com/KaviyaGnanavel/Movie-reccomendation/raw/refs/heads/main/u.data', sep='\t', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Merge to get movie titles with ratings
movie_ratings = pd.merge(ratings, movies, on='movie_id')

# Create a simple content column using titles (not ideal, but okay for beginner level)
movies['content'] = movies['title']

# Use TF-IDF to vectorize titles (can be improved later)
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['content'])

# Compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Build a reverse map of indices and movie titles
indices = pd.Series(movies.index, index=movies['title'])

# Recommendation function
def get_recommendations(title, cosine_sim=cosine_sim):
    # Normalize input title: strip spaces and lowercase
    title = title.strip().lower()

    # Normalize dataset titles similarly
    indices_lower = pd.Series(movies.index, index=movies['title'].str.lower().str.strip())

    idx = indices_lower.get(title)
    if idx is None:
        print("Movie not found. Please check spelling or include the year (e.g. 'Star Wars (1977)').")
        return []
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 5 similar movies
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]

# Test it
print("Recommendations for 'Star Wars (1977)':")
print(get_recommendations('Star Wars (1977)'))

Recommendations for 'Star Wars (1977)':
513        Annie Hall (1977)
666       Audrey Rose (1977)
1090    Pete's Dragon (1977)
1292         Star Kid (1997)
123         Lone Star (1996)
Name: title, dtype: object


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create input box and button
movie_input = widgets.Text(
    value='',
    placeholder='Type a movie title here',
    description='Movie:',
    disabled=False
)
button = widgets.Button(description="Get Recommendations")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        title = movie_input.value
        recs = get_recommendations(title)
        if len(recs) == 0:
            print("Movie not found. Try another one.")
        else:
            print(f"Top recommendations similar to '{title}':")
            for movie in recs:
                print(f"- {movie}")

button.on_click(on_button_clicked)

display(movie_input, button, output)

Text(value='', description='Movie:', placeholder='Type a movie title here')

Button(description='Get Recommendations', style=ButtonStyle())

Output()